Source: ICLR

Year: 2016

Authors: Tom Schaul, John Quan, Ioannis Antonoglou, David Silver

Institutions: Google DeepMind

Prioritized Experience Replay replays transitions with high expected learning progress, as measured by the magnitude of their temporal-difference (TD) error. The prioritization can lead to a loss of diversity, which is alleviated with stochastic prioritization, and introduce bias, which is corrected with importance sampling.